In [30]:
import pandas as pd
import numpy as np

In [31]:
data = pd.read_csv('Fish Data - A2.csv')

mean = data.mean()
std = data.std()

In [32]:
mean, std

(Height      8.970994
 Width       4.417486
 Weight    398.326415
 dtype: float64,
 Height      4.286208
 Width       1.685804
 Weight    357.978317
 dtype: float64)

In [33]:
data.isna().sum()

Height    0
Width     0
Weight    0
dtype: int64

### Preprocessing

In [34]:
data.fillna(data.mean(), inplace=True)

In [35]:
mean_ht = data['Height'].mean()
mean_wd = data['Width'].mean()
std_ht = data['Height'].std()
std_wd = data['Width'].std()

mean_ht, mean_wd, std_ht, std_wd

(8.970993710691824, 4.417485534591195, 4.286207619968867, 1.6858038699921671)

In [36]:
data['Width'] = (data['Width'] - mean_wd) / std_wd
data['Height'] = (data['Height'] - mean_ht) / std_ht

In [37]:
ht = data['Height'].values
wd = data['Width'].values
wt = data['Weight'].values

In [38]:
len(ht) == len(wd), len(wt), len(wd)

(True, 159, 159)

In [45]:
train_size = int(0.8 * len(data))

train_data = data[:train_size]
test_data = data[train_size:]

X_train = train_data[['Height', 'Width']].values
y_train = train_data['Weight'].values
X_test = test_data[['Height', 'Width']].values
y_test = test_data['Weight'].values

In [40]:
len(train_data), len(test_data)

(127, 32)

In [41]:
degrees = [_ for _ in range(10)]

degrees

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [42]:
train_data_ft = {}

In [44]:
def create_polynomial_features(X, degree):
    X_pol = np.column_stack([X[:, 0] ** i * X[:, 1] ** (degree - i) for i in range(degree + 1)])
    return X_pol

def fit_polynomial_regression(X, y):
    X_transpose = X.T
    w = np.linalg.inv(X_transpose.dot(X)).dot(X_transpose).dot(y)
    return w

def calculate_mse(w, X, y):
    y_pred = X.dot(w)
    mse = np.mean((y_pred - y) ** 2)
    return mse

def regularized_loss(X, y, w, q, lambda_reg):
    n = len(y)
    predictions = np.dot(X, w)
    error = predictions - y
    loss = (1 / (2 * n)) * np.sum(np.abs(error) ** q) + (lambda_reg / 2) * np.dot(w, w)
    return loss


# def select_best_degree(train_data_pol, test_data_pol, train_target, test_target):
#     best_degree = None
#     best_mse = float('inf')
    
#     for degree, train_poly_features in train_data_pol.items():
#         model = fit_polynomial_regression(train_poly_features, train_target)
#         test_poly_features = test_data_pol[degree]
#         mse = calculate_mse(model, test_poly_features, test_target)
        
#         if mse < best_mse:
#             best_mse = mse
#             best_degree = degree
    
#     return best_degree

def batch_gradient_descent_regularized(X, y, w, lr, num_iters, q, lambda_reg):
    n = len(y)
    losses = []
    
    for _ in range(num_iters):
        predictions = np.dot(X, w)
        error = predictions - y
        gradient = (1 / n) * X.T.dot(np.abs(error) ** (q - 1) * np.sign(error)) + lambda_reg * w
        w -= lr * gradient
        loss = regularized_loss(X, y, w, q, lambda_reg)
        losses.append(loss)
    
    return w, losses

def stochastic_gradient_descent_regularized(X, y, w, lr, num_iters, q, lambda_reg):
    n = len(y)
    losses = []
    
    for _ in range(num_iters):
        rand_idx = np.random.randint(n)  
        x_i = X[rand_idx, :]
        y_i = y[rand_idx]
        
        prediction = np.dot(x_i, w)
        error = prediction - y_i
        gradient = x_i * (np.abs(error) ** (q - 1) * np.sign(error)) + lambda_reg * w
        w -= lr * gradient
        loss = regularized_loss(X, y, w, q, lambda_reg)
        losses.append(loss)
    
    return w, losses

In [ ]:
q = [0.5,1,2,4]
lambda_values = np.linspace(0, 1, 11)

lr = 0.005
num_iters = 5000

best_models_batch = {}
best_models_stochastic = {}

for q in q:
    best_loss_batch = float('inf')
    best_loss_stochastic = float('inf')
    
    for lambda_reg in lambda_values:
        # Create polynomial features for training and testing data
        train_data_poly = create_polynomial_features(X_train, best_degree)
        test_data_poly = create_polynomial_features(X_test, best_degree)
        
        # Train batch gradient descent model
        w_batch, losses_batch = batch_gradient_descent_regularized(train_data_poly, y_train, lr, num_iters, q, lambda_reg)
        final_loss_batch = losses_batch[-1]
        
        # Train stochastic gradient descent model
        w_stochastic, losses_stochastic = stochastic_gradient_descent_regularized(train_data_poly, y_train, lr, num_iters, q, lambda_reg)
        final_loss_stochastic = losses_stochastic[-1]
        
        if final_loss_batch < best_loss_batch:
            best_loss_batch = final_loss_batch
            best_models_batch[(q, lambda_reg)] = w_batch
        
        if final_loss_stochastic < best_loss_stochastic:
            best_loss_stochastic = final_loss_stochastic
            best_models_stochastic[(q, lambda_reg)] = w_stochastic